<a href="https://colab.research.google.com/github/skyjadel/chang-s_titanic/blob/main/Task_02_%E4%BD%9C%E6%A5%AD_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task #02 作業#01【實作題】鐵達尼號存活預測專題實作（1） - 環境準備
完成後也請你繳交今日的作業，直接複製底下格式填上內容、標題設定成「2024 Task#02 作業 01」：

這次的範例是一個專案等級的題目，我們建議可以使用 `Github` + `Colab` 進行操作。Github 是一個程式碼的雲端託管平台，Repository 是一個專案單位，我們能夠利用 Github 當成是簡單的程式碼作品集。Colab 是由 Google 維護的類似 Jupyter 環境整合式開發環境，提供在雲端網頁上執行進行 Python 程式。Colab 或 Jupyter 能夠以互動式程式執行環境，每次執行為一個區塊，適合像資料分析、資料探索需要大量互動操作資料的任務。可以依照以下步驟開始建立你的專案：

* 建立一個 GitHub 帳號，建立一個的「Repository」專案，存放這次練習的 實作專案

* 建立一個 Google Colab 程式碼，開始這次實作練習

In [1]:
import numpy as np
import pandas as pd